# Using `sqlite_tools` to Query Our SQLite database containing our copy of the NZGD

This notebook demonstrates how to interact with our SQLite database containing our copy of the NZGD. We'll cover:
1.  Connecting to an SQLite database.
2.  Basic SQLite query structure.
3.  Using the functions provided in the `sqlite_tools.query_sqlite_db` module to extract specific data.

## 1. Accessing SQLite Database from Python

Python's built-in `sqlite3` module allows us to connect to and interact with SQLite databases. 

First, you need to import the necessary libraries: `sqlite3` for database interaction, `pathlib` for handling file paths, and `pandas` for working with data in DataFrames. If you plan to use the `sqlite_tools` package, you'll import its functions as well.

In [2]:
import sqlite3
from pathlib import Path

import pandas as pd

from sqlite_tools import query

Next, specify the path to your SQLite database file. **Remember to replace `"/path/to/your/nzgd_database.db"` with the actual path to your database file.**

In [ ]:
# Define the path to your SQLite database
# !!! IMPORTANT: Replace this with the actual path to your database file !!!
db_file_path = Path("/path/to/your/nzgd_database.db")

# Establish a database connection
conn = sqlite3.connect(db_file_path)
print(f"Successfully connected to {db_file_path}")

# Note: The connection `conn` will be used in subsequent cells.
# We will close it at the end of the notebook.

Successfully connected to /home/arr65/data/nzgd/extracted_nzgd_data_until_8_Nov_2024.db


## 2. Understanding SQLite Query Structure

SQL (Structured Query Language) is used to communicate with databases. SQLite uses a dialect of SQL. Here are some common clauses:

*   **`SELECT column1, column2, ... FROM table_name`**: This is the most fundamental clause. It retrieves specified columns from a table.
    *   `SELECT * FROM table_name` selects all columns.
*   **`WHERE condition`**: Filters records based on a specific condition. 
    *   Example: `WHERE age > 30` or `WHERE name = 'Alice'`.
*   **`JOIN another_table ON table_name.column_name = another_table.column_name`**: Combines rows from two or more tables based on a related column between them.
    *   Common types: `INNER JOIN` (default), `LEFT JOIN`.
*   **`ORDER BY column_name [ASC|DESC]`**: Sorts the result set by one or more columns, either in ascending (`ASC`, default) or descending (`DESC`) order.
*   **`LIMIT number`**: Restricts the number of rows returned by the query.
*   **`GROUP BY column_name`**: Groups rows that have the same values in specified columns into summary rows. Often used with aggregate functions like `COUNT()`, `MAX()`, `MIN()`, `SUM()`, `AVG()`.

**Example of a more complex query structure:**

```sql
SELECT 
    t1.columnA, 
    t2.columnB, 
    COUNT(t1.id) as item_count
FROM 
    table1 AS t1
INNER JOIN 
    table2 AS t2 ON t1.common_id = t2.common_id
WHERE 
    t1.status = 'active' AND t2.category = 'electronics'
GROUP BY 
    t1.columnA, t2.columnB
ORDER BY 
    item_count DESC
LIMIT 10;
```
This query selects data from `table1` and `table2`, filters it, groups it, counts items, orders the result, and limits it to the top 10.

## 3. Using Functions from `sqlite_tools.query`

The `sqlite_tools` package provides functions to easily extract some data. These functions encapsulate SQL queries.

**Ensure your database connection (`conn`) is active from Cell 2.**

### Example 1: Get measurements for a specific CPT using its NZGD ID

In [4]:
nzgd_id_for_cpt = 1 

if conn:
    cpt_df = query.cpt_measurements_for_one_nzgd(selected_nzgd_id=nzgd_id_for_cpt, conn=conn)
    if not cpt_df.empty:
        display(cpt_df.head())
    else:
        print(f"No CPT measurements found for NZGD ID {nzgd_id_for_cpt}.")
else:
    print("Database connection is not established.")

Time to extract CPT measurements for cpt_id=1 from SQLite: 0.01 s


,depth,qc,fs,u2,cpt_id,nzgd_id
0,0.00,0.05,0.000,0.000,0,1
1,0.01,0.09,0.000,0.001,0,1
2,0.02,0.21,0.000,0.000,0,1
3,0.03,0.27,0.000,0.000,0,1
4,0.04,0.28,0.001,0.000,0,1


### Example 2: Get measurements for a specific SPT using its NZGD ID

In [5]:
nzgd_id_for_spt = 14810

if conn:
    spt_df = query.spt_measurements_for_one_nzgd(selected_nzgd_id=nzgd_id_for_spt, conn=conn)
    if not spt_df.empty:
        display(spt_df.head())
    else:
        print(f"No SPT measurements found for NZGD ID {nzgd_id_for_spt}.")
else:
    print("Database connection is not established.")

Time to extract CPT measurements for cpt_id=14810 from SQLite: 0.00 s


,depth,n,nzgd_id
0,1.0,2,14810
1,2.0,0,14810
2,3.0,14,14810
3,4.0,21,14810
4,5.0,29,14810


### Example 3: Get Soil Type measurements for a specific SPT using its NZGD ID

In [7]:
nzgd_id_for_spt_soil = 14810

if conn:
    spt_soil_df = query.spt_soil_types_for_one_nzgd(selected_nzgd_id=nzgd_id_for_spt_soil, conn=conn)
    if not spt_soil_df.empty:
        display(spt_soil_df.head())
    else:
        print(f"No SPT soil types found for NZGD ID {nzgd_id_for_spt_soil}.")
else:
    print("Database connection is not established.")

,top_depth,nzgd_id,soil_type,layer_thickness
0,0.0000,14810,SILT,0.3095
1,0.3095,14810,SILT,2.6905
2,3.0000,14810,SAND,0.3105
3,3.3105,14810,SAND,0.6895
4,4.0000,14810,SAND,0.4015


### Example 4: Get Vs30 estimates for a specific CPT investigation given its NZGD ID

In [8]:
nzgd_id_for_cpt_vs30 = 1

if conn:
    cpt_vs30_df = query.cpt_vs30s_for_one_nzgd_id(selected_nzgd_id=nzgd_id_for_cpt_vs30, conn=conn)
    if not cpt_vs30_df.empty:
        display(cpt_vs30_df.head())
    else:
        print(f"No CPT Vs30s found for NZGD ID {nzgd_id_for_cpt_vs30}.")
else:
    print("Database connection is not established.")

Time to extract Vs30s for nzgd_id=1 from SQLite: 0.00 s


,cpt_id,nzgd_id,vs30,vs30_stddev,cpt_file,cpt_tip_net_area_ratio,measured_gwl,deepest_depth,shallowest_depth,cpt_to_vs_correlation,...,model_vs30_foster_2019,model_vs30_stddev_foster_2019,model_gwl_westerhoff_2019,region,district,city,suburb,record_name,vs30_log_residual,gwl_residual
0,0,1,278.243722,0.014691,CPT_1_AGS01.xls-TabulatedData,None,1.0,26.04,0.0,andrus_2007_pleistocene,...,219.36679,0.233388,4.740686,Canterbury,Christchurch_City,Christchurch,Aranui,CPT_1,0.237752,-3.740686
1,0,1,226.222757,0.014691,CPT_1_AGS01.xls-TabulatedData,None,1.0,26.04,0.0,andrus_2007_holocene,...,219.36679,0.233388,4.740686,Canterbury,Christchurch_City,Christchurch,Aranui,CPT_1,0.030775,-3.740686
2,0,1,583.194908,0.014691,CPT_1_AGS01.xls-TabulatedData,None,1.0,26.04,0.0,andrus_2007_tertiary_age_cooper_marl,...,219.36679,0.233388,4.740686,Canterbury,Christchurch_City,Christchurch,Aranui,CPT_1,0.977776,-3.740686
3,0,1,229.364171,0.014691,CPT_1_AGS01.xls-TabulatedData,None,1.0,26.04,0.0,robertson_2009,...,219.36679,0.233388,4.740686,Canterbury,Christchurch_City,Christchurch,Aranui,CPT_1,0.044566,-3.740686
4,0,1,236.463482,0.014691,CPT_1_AGS01.xls-TabulatedData,None,1.0,26.04,0.0,hegazy_2006,...,219.36679,0.233388,4.740686,Canterbury,Christchurch_City,Christchurch,Aranui,CPT_1,0.075049,-3.740686


### Example 5: Get Vs30 Estimates for a specific SPT investigation given its NZGD ID

In [9]:
nzgd_id_for_spt_vs30 = 14810

if conn:
    spt_vs30_df = query.spt_vs30s_for_one_nzgd_id(selected_nzgd_id=nzgd_id_for_spt_vs30, conn=conn)
    if not spt_vs30_df.empty:
        display(spt_vs30_df.head())
    else:
        print(f"No SPT Vs30s found for NZGD ID {nzgd_id_for_spt_vs30}.")
else:
    print("Database connection is not established.")

Time to extract CPT measurements for cpt_id=14810 from SQLite: 0.00 s
Time to extract Vs30s for nzgd_id=14810 from SQLite: 0.00 s


,nzgd_id,spt_borehole_diameter_for_vs30_calculation,vs30,vs30_stddev,spt_vs30_calculation_used_efficiency,spt_vs30_calculation_used_soil_info,borehole_file,spt_efficiency,spt_borehole_diameter,measured_gwl,...,hammer_type,region,district,city,suburb,deepest_depth,shallowest_depth,record_name,vs30_log_residual,gwl_residual
0,14810,150.0,216.479501,4.310376,0,0,Borehole_14810_AGS01.ags,None,None,None,...,Auto,Canterbury,Christchurch_City,Christchurch,Belfast,15.0,1.0,BH_14810,-0.014485,NaN
1,14810,150.0,209.681191,4.294176,0,0,Borehole_14810_AGS01.ags,None,None,None,...,Safety,Canterbury,Christchurch_City,Christchurch,Belfast,15.0,1.0,BH_14810,-0.046393,NaN
2,14810,150.0,193.369746,4.253128,0,0,Borehole_14810_AGS01.ags,None,None,None,...,Standard,Canterbury,Christchurch_City,Christchurch,Belfast,15.0,1.0,BH_14810,-0.127377,NaN
3,14810,150.0,186.863819,0.045925,0,0,Borehole_14810_AGS01.ags,None,None,None,...,Auto,Canterbury,Christchurch_City,Christchurch,Belfast,15.0,1.0,BH_14810,-0.161601,NaN
4,14810,150.0,181.068634,0.045925,0,0,Borehole_14810_AGS01.ags,None,None,None,...,Safety,Canterbury,Christchurch_City,Christchurch,Belfast,15.0,1.0,BH_14810,-0.193105,NaN


### Example 6: Get All Estimated Vs30s Given Specific Correlations

Please refer to [available_options.md](./available_options.md) to see the available correlation options.



In [10]:
vs30_corr = "boore_2004"
cpt_vs_corr = "andrus_2007_pleistocene"
spt_vs_corr = "brandenberg_2010"
hammer = "Auto"

if conn:

    all_vs30_data_df = query.all_vs30s_given_correlations(
        selected_vs_to_vs30_correlation=vs30_corr,
        selected_cpt_to_vs_correlation=cpt_vs_corr,
        selected_spt_to_vs_correlation=spt_vs_corr,
        selected_hammer_type=hammer,
        conn=conn
    )
    if not all_vs30_data_df.empty:
        display(all_vs30_data_df.head())
    else:
        print("No Vs30 data found for the given correlations.")
else:
    print("Database connection is not established.")

Time to extract CPT Vs30s and metadata from SQLite: 0.40 s
Time to extract SPT Vs30s and metadata from SQLite: 0.03 s


,cpt_id,nzgd_id,vs30,vs30_stddev,type_prefix,original_reference,investigation_date,published_date,latitude,longitude,...,region,district,suburb,city,record_name,vs30_log_residual,gwl_residual,spt_efficiency,spt_borehole_diameter,type_number_code
0,0.0,1,278.243722,0.014691,CPT,CPT-ARN-01,2010-11-19,2012-06-13,-43.521206,172.708811,...,Canterbury,Christchurch_City,Aranui,Christchurch,CPT_1,0.237752,-3.740686,NaN,NaN,0
1,1.0,2,274.746826,0.000000,CPT,CPT-ARN-02,2010-11-19,2012-06-13,-43.512666,172.705791,...,Canterbury,Christchurch_City,Aranui,Christchurch,CPT_2,0.254764,-1.034373,NaN,NaN,0
2,2.0,3,250.001972,0.002736,CPT,CPT-ARN-03,2010-11-19,2012-06-13,-43.512321,172.709380,...,Canterbury,Christchurch_City,Aranui,Christchurch,CPT_3,0.220879,-2.037427,NaN,NaN,0
3,3.0,4,248.629569,0.000000,CPT,CPT-ARN-04,2010-11-19,2012-06-13,-43.511633,172.712160,...,Canterbury,Christchurch_City,Aranui,Christchurch,CPT_4,0.205407,-0.732629,NaN,NaN,0
4,4.0,5,NaN,NaN,CPT,CPT-ARN-05,2010-11-19,2012-06-13,-43.510262,172.714002,...,Canterbury,Christchurch_City,Aranui,Christchurch,CPT_5,NaN,-0.625690,NaN,NaN,0


## Closing the Connection

Finally, it's important to close the database connection when you're done with it to free up system resources.

In [11]:
if conn: # if a connection exists
    conn.close()
    print("\nDatabase connection closed.")
else:
    print("No database connection to close.")


Database connection closed.
